# Load NJ Payroll (Jan 15, 2019) Data from csv to postgresql from python

In [2]:
import pandas as pd
import psycopg2
import pprint as pp
import utils

In [3]:
#!wget -O raw_data/YourMoney_Agency_Payroll.csv https://data.nj.gov/api/views/iqwc-r2w7/rows.csv?accessType=DOWNLOAD

In [4]:
df = pd.read_csv('raw_data/YourMoney_Agency_Payroll.csv')
df['AS_OF_DATE'] = pd.to_datetime(df['AS_OF_DATE'], format='%B %d %Y')
df = utils.clean(df)

In [8]:
# for quick loading back to pandas with data compression
df.to_parquet('processed_data/YourMoney_Agency_Payroll.parquet', compression='GZIP')

In [ ]:
# or use hdf if you prefer faster loading but less compression
df.to_hdf('processed_data/YourMoney_Agency_Payroll.hdf','njpayroll')

In [ ]:
'''
# You can also load the data from the API. You need to create an apptoken so you aren't throttled.
from sodapy import Socrata
client = Socrata("data.nj.gov", None)
apptoken = 'APPTOKENHERE'
results = client.get("swjq-h2b6", apptoken, limit=2000000) # Limit set to 2000000 as default is only 1000
df = pd.DataFrame.from_records(results)
'''

In [ ]:
df.dtypes # Checking dtypes to make sure they look ok

# Create new database, create new table within the new database, load data into new table.

In [29]:
conn = psycopg2.connect(dbname='postgres', user='postgres', password='postgres')
conn.autocommit = True
cur = conn.cursor()
cur.execute('create database nj_payroll owner jk')

In [31]:
df_cols = {}
for col in df.columns:
    df_cols[col] = str(df[col].dtype)
remap = {'int64': 'numeric', 'datetime64[ns]': 'date', 'float64': 'numeric', 'object': 'text'}
for k, v in df_cols.items():
    df_cols[k] = remap[v]
query = 'create table if not exists nj_payroll ('+', '.join([' '.join([k,v]) for k, v in df_cols.items()]) + ')'

In [6]:
conn = psycopg2.connect(dbname='nj_payroll', user='jk')
conn.autocommit = True
cur = conn.cursor()
query = 'create table if not exists nj_payroll (CALENDAR_YEAR numeric, CALENDAR_QUARTER numeric, AS_OF_DATE date, PAYROLL_ID numeric, LAST_NAME text, FIRST_NAME text, MIDDLE_INITIAL text, FULL_NAME text, SALARY_HOURLY_RATE numeric, MASTER_DEPARTMENT_AGENCY_DESC text, MASTER_SECTION_DESC text, MASTER_TITLE_DESC text, EMPLOYEE_RELATIONS_GROUP text, COMPENSATION_METHOD text, MASTER_YTD_REGULAR_PAY numeric, MASTER_YTD_OVERTIME_PAYMENTS numeric, MASTER_YTD_ALL_OTHER_PAYMENTS numeric, MASTER_YTD_EARNINGS numeric, PAID_DEPARTMENT_AGENCY_DESC text, PAID_SECTION_DESC text, REGULAR_PAY numeric, SUPPLEMENTAL_PAY numeric, ONE_TIME_PAYMENTS numeric, LEGISLATOR_OR_BACK_PAY numeric, OVERTIME_PAYMENTS numeric, CLOTHING_UNIFORM_PAYMENTS numeric, RETROACTIVE_PAY numeric, LUMP_SUM_PAY numeric, CASH_IN_LIEU_MAINTENANCE numeric, YTD_EARNINGS numeric, RECORD_TYPE text)'
cur.execute(query)

In [25]:
# doing this way because df.to_sql errors out for some reason
import csv
from datetime import datetime

conn = psycopg2.connect(dbname='nj_payroll', user='jk')
cur = conn.cursor()
i = 0
with open('YourMoney_Agency_Payroll.csv', 'r') as f:
    reader = csv.reader(f)
    next(f)
    for row in reader:
        update = row[0:2]
        update.append(datetime.strptime(row[2], '%B %d %Y').date())
        update.extend(row[3:])
        update = [None if i == '' else i for i in update]
                
        cur.execute('''insert into nj_payroll values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''', update)
        i += 1
        if i % 100000 == 0:
            print(i, 'entries done')
    conn.commit()

100000 entries done
200000 entries done
300000 entries done
400000 entries done
500000 entries done
600000 entries done
700000 entries done
800000 entries done
900000 entries done
1000000 entries done
1100000 entries done
1200000 entries done
1300000 entries done
1400000 entries done
1500000 entries done


In [26]:
cur.execute('''select * from nj_payroll limit 10''')
pp.pprint(cur.fetchall())

[(Decimal('2018'),
  Decimal('4'),
  datetime.date(2018, 12, 31),
  Decimal('1'),
  'REBER',
  'FREDERICK',
  None,
  'REBER, FREDERICK',
  Decimal('25.69'),
  'CORRECTIONS',
  'SOUTH WOODS STATE PRISON',
  'SPECIAL SERVICES',
  'SPECIAL SERVICES - UNIT Q',
  'HOURLY',
  Decimal('12652.38'),
  Decimal('0.00'),
  Decimal('2965.95'),
  Decimal('15618.33'),
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'MASTER'),
 (Decimal('2018'),
  Decimal('4'),
  datetime.date(2018, 12, 31),
  Decimal('1'),
  'REBER',
  'FREDERICK',
  None,
  'REBER, FREDERICK',
  None,
  'CORRECTIONS',
  'SOUTH WOODS STATE PRISON',
  'SPECIAL SERVICES',
  'SPECIAL SERVICES - UNIT Q',
  'HOURLY',
  None,
  None,
  None,
  None,
  'CORRECTIONS',
  'SOUTH WOODS STATE PRISON',
  Decimal('12652.38'),
  Decimal('0.00'),
  Decimal('650.00'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('2315.95'),
  Decimal('0.00'),
  Decimal('0.00'),
  Decimal('15618.